In [21]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from stumpy import mstump
import glob
import ipywidgets as widgets
import h5py
import numpy as np
from tslearn.clustering import TimeSeriesKMeans

main_df = pd.read_pickle("/share/data/temp/athira/July17_features_combined_noLightStimuli.pickle")
motif_df = pd.read_hdf("/work/daniel/matrix_profile_curvature/matrix_profile_motifs/labeled_motifs_attempt2/with_speed/correct_frame_motifs_30_nclusters_15_meanvarnorm_rollingmean_lowthres_withsomespeed.hdf5")
    
#     "correct_frame_motifs_30_nclusters_10_meanvarnorm_rollingmean_labeled.hdf5")

files = motif_df.filename.unique()
raw_skel_files = glob.glob("/share/data/temp/athira/Tierpsy_skeleton_files_July13/*skeletons.hdf5")

In [19]:
model = TimeSeriesKMeans.from_json(f"model_with_context_n25_w30_withspeed_softdtw.json")

In [23]:
motif_df["labels_softdtw_25"] = model.labels_

In [24]:
motif_df.columns

Index(['filename', 'motifs', 'frame', 'labels_dtw', 'labels_softdtw', 'drugs',
       'labels_softdtw_25'],
      dtype='object')

In [25]:
%matplotlib widget

In [26]:
def read_skeleton_as_array(file_path):
    file_path = list(filter(lambda x: file_path in x, raw_skel_files))[0]
    try:
        skeletons_store = h5py.File(file_path, 'r')
        skeletons = np.array(skeletons_store['coordinates']['skeletons'])

    except IOError:
        print("Unable to read the file : check if the file exists")

    except KeyError:
        skeletons = np.array(skeletons_store['skeleton'])

    return skeletons

In [27]:
motif_df.iloc[0]

filename             20180919_115015_1_5m0s_None_None_None_INVERTED...
motifs               [[-0.008312709326855839, 0.011686712922528385,...
frame                                                               24
labels_dtw                                                          11
labels_softdtw                                                       6
drugs                                                             None
labels_softdtw_25                                                   19
Name: 0, dtype: object

In [28]:
skel = read_skeleton_as_array(motif_df.iloc[0].filename)

In [29]:
def rotate(x, y, angle):
    xr = x*np.cos(angle) - y* np.sin(angle)
    yr = x*np.sin(angle) + y* np.cos(angle)
    return xr,yr
def get_base_angle(x,y, degrees = False):
    angle = np.arctan(y/x)
    if y < 0:
        angle *=-1
    if not degrees:
        return(angle)
    else:
        return(np.degrees(angle))

In [30]:
def get_skeleton_motif(motif):
    skel = read_skeleton_as_array(motif.filename)
    start = motif.frame
    stop = motif.frame + 30
    skel =  skel[start:stop, :,:]
    return(skel)

In [31]:
from matplotlib.animation import FuncAnimation as animation
from IPython.display import HTML

In [32]:
plt.close("all")

In [33]:

def animate(i):
#     if i%30 == 0:
#         ax.clear()
    if i < len(m):
        ax.clear()
        frame = m[i, :,:]
        xs = frame[:,0]
        ys = frame[:,1]
        angle = get_base_angle(xs[12], ys[12]) + np.pi
        xs = xs-frame[12, 0]
        ys = ys-frame[12, 1]

        xs, ys = rotate(xs, ys, angle)
    #     frame = frame - m[0][10]
    #     x, y = frame = rotate(frame[:,0], frame[:,1], angle)

        s, = ax.plot(xs, ys, c = "b", alpha = 0.4, lw = 2)
#         ax.set_xlim(frame[0,0]-100, frame[0,0]+100)
#         ax.set_ylim(ys[0]-100, ys[0]+100)
        ax.set_title(f"motif {get}, cluster={cluster}")
        return(s)



In [34]:
fig, ax = plt.subplots()
anim = animation(fig, animate, frames = 30, interval = 100)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [35]:
plt.show()

In [36]:
import tqdm

In [37]:
plt.close("all")

In [38]:
for cluster in tqdm.tqdm(range(15)):
    twelves = motif_df[motif_df["labels_softdtw_25"] == cluster]
    gets = np.random.choice(np.arange(0,len(twelves),1), 5)
    for get in gets:
        m = get_skeleton_motif(twelves.iloc[get])
        fig, ax = plt.subplots()
        anim = animation(fig, animate, frames = 30, interval = 100).save(f"models_to_compare/gifs/animate_cluster_{cluster}_motif_{get}.gif", writer = "imagemagick")
        plt.close(fig)

  0%|          | 0/15 [00:00<?, ?it/s]

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

MovieWriter stderr:
convert-im6.q16: unable to open image `models_to_compare/gifs/animate_cluster_0_motif_1072.gif': No such file or directory @ error/blob.c/OpenBlob/2701.

  0%|          | 0/15 [00:02<?, ?it/s]


CalledProcessError: Command '['convert', '-size', '640x480', '-depth', '8', '-delay', '10.0', '-loop', '0', 'rgba:-', 'models_to_compare/gifs/animate_cluster_0_motif_1072.gif']' returned non-zero exit status 1.

In [11]:
def animate(i):
    subplots = axes.ravel()
#     if i%30 == 0:
#         ax.clear()
    to_return = []
    for j, m in enumerate(selected_motifs):
        print(j)
        if i < len(m):
            if i%30 == 0:
                subplots[j].clear()
            
            frame = m[i, :,:]
            xs = frame[:,0]
            ys = frame[:,1]
            angle = get_base_angle(xs[12], ys[12]) - np.pi
            xs = xs-frame[12, 0]
            ys = ys-frame[12, 1]

            xs, ys = rotate(xs, ys, angle)
            s, = subplots[j].plot(xs, ys, c = "b", alpha = 0.4, lw = 2)
            subplots[j].set_title(f"motif {get}, cluster={cluster}")
            to_return.append(s)
        return(to_return)

In [12]:
plt.close("all")

In [29]:
import tqdm

In [34]:
def animate(i):
    fig.suptitle(f"softdtw_cluster_{cluster}")
    for j, m in enumerate(selected_motifs):
        if i%30 == 0:
            axes.ravel()[j].clear()
        frame = m[i,:,:]
        xs = frame[:,0]
        ys = frame[:,1]
        angle = get_base_angle(xs[12], ys[12]) - np.pi
        xs = xs-frame[12, 0]
        ys = ys-frame[12, 1]
        xs, ys = rotate(xs, ys, angle)
        axes.ravel()[j].plot(xs, ys, c = "b", alpha = 0.2)
        axes.ravel()[j].axis("off")
        
        


for cluster in tqdm.tqdm(range(15)): 
    twelves = motif_df[motif_df["labels_softdtw"] == cluster]
    gets = np.random.choice(np.arange(0,len(twelves),1), 225)
    selected_motifs = [get_skeleton_motif(twelves.iloc[get]) for get in gets]
    fig, axes = plt.subplots(15,15, figsize = (10,10))

    anim = animation(fig, animate, frames = 30, interval = 100).save(f"models_to_compare/gifs/softdtw_30_15_{str(cluster).zfill(2)}.gif", writer = "imagemagick")

  0%|          | 0/15 [00:00<?, ?it/s]/share/python-virtual-environments/daniel-jl/lib/python3.6/site-packages/ipykernel_launcher.py:23: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

  7%|▋         | 1/15 [00:42<09:56, 42.62s/it]/share/python-virtual-environments/daniel-jl/lib/python3.6/site-packages/ipykernel_launcher.py:23: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

 13%|█▎        | 2/15 [02:20<12:49, 59.20s/it]/share/python-virtual-environments/daniel-jl/lib/python3.6/site-packages/ipykernel_launcher.py:23: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


...

 87%|████████▋ | 13/15 [10:55<01:26, 43.07s/it]/share/python-virtual-environments/daniel-jl/lib/python3.6/site-packages/ipykernel_launcher.py:23: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

 93%|█████████▎| 14/15 [11:35<00:42, 42.06s/it]/share/python-virtual-environments/daniel-jl/lib/python3.6/site-packages/ipykernel_launcher.py:23: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

100%|██████████| 15/15 [12:36<00:00, 50.44s/it]
